In [41]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [42]:
groups_file = open('../data/groups', '+r')
artists = groups_file.readlines()
artists = list(map(str.strip, artists))
df_n = pd.DataFrame(columns=['group','song_lines'])



In [43]:

for name in range(len(artists)):
  
    df = pd.read_json(artists[name]+'.json', typ='series')
    df  = df.reset_index()
    s_len = len(df.index)
    df['group']=[artists[name] for i in range(s_len)]
  
    df = df.rename(columns={"index":"song_name", 0: "song_text"})
    df['song_lines'] = df[['song_text']]
    df['song_lines'] = df['song_lines'].apply(lambda x: x.lower().split('\n'))

    df = df.explode('song_lines')
    df = df[df['song_lines'].astype(bool)]
    print(artists[name], df['group'].unique(), df.shape)
    df_n = pd.concat([df_n, df[['group','song_lines']]], ignore_index=True)
    
print("Main dataframe: ", df_n.info())
 

Kaleo ['Kaleo'] (595, 4)
Måneskin ['Måneskin'] (2006, 4)
Coldplay ['Coldplay'] (10623, 4)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13224 entries, 0 to 13223
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   group       13224 non-null  object
 1   song_lines  13224 non-null  object
dtypes: object(2)
memory usage: 206.8+ KB
Main dataframe:  None


In [40]:
df_n.to_csv('../data/music_3.csv')

# Pipeline

#### Tfildvectorizer
#### PCA
#### SMOTE
#### NaiveBayers

In [1]:
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB
import imblearn
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, MiniBatchNMF, LatentDirichletAllocation
from time import time
from sklearn.ensemble import RandomForestClassifier

In [52]:
count_vectorizer = CountVectorizer(
    #
    ###################################################################
    # convert all characters to lowercase before tokenizing
    #
    lowercase = True, 
    #
    ###################################################################
    # choose words to create features
    #
    analyzer = 'word',
    #
    ###################################################################
    # use built-in stop word list for English (default=None)
    #
    stop_words = 'english',
    #
    ###################################################################
    # select tokens of 2 or more word characters (punctuation ignored) 
    #
    token_pattern = r"(?u)\b\w\w+\b",
    #
    ###################################################################
    # consider only unigrams of tokens
    #
    ngram_range = (5, 5)
    #
    ###################################################################  
)

In [53]:
vec = count_vectorizer.fit_transform(df_n['song_lines'])
count_vectorizer.get_feature_names_out()

array(['15 minuti perché spaccare tutto', '17 brah ya ah don',
       'abbiamo da giovani una bellissima', ...,
       'zitto invece adesso guardami quindi',
       'él negro fuerte bronco como', 'él se murió mi amigo'],
      dtype=object)

In [58]:
count_matrix_skl = pd.DataFrame(
    vec.todense(), 
    columns=count_vectorizer.get_feature_names_out(), 
    index=df_n['group']
)

In [59]:
count_matrix_skl 

,15 minuti perché spaccare tutto,17 brah ya ah don,abbiamo da giovani una bellissima,acabar mientras le decía despacito,accanto te baby accanto te,accanto te io morirò da,acqua da bere il significato,acqua del mare che le,acqua forse il mare dentro,act north west east south,...,young singer writer like gabriella,zattera che naviga fiume piena,zelante capelli come rose rosse,zero da dimostrarvi non sono,zero ma lo zero piscia,zero piscia testa te il,zitti buoni qui la gente,zitto invece adesso guardami quindi,él negro fuerte bronco como,él se murió mi amigo
group,,,,,,,,,,,,,,,,,,,,,
Kaleo,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Kaleo,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Kaleo,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Kaleo,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Kaleo,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Måneskin,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Måneskin,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Måneskin,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [99]:
# feature and target columns
X = count_matrix_skl.reset_index().drop(['group'], axis=1)
y = count_matrix_skl.reset_index()['group']
X.shape, y.shape

((1813, 2930), (1813,))

In [110]:
# instanciation
vectorizer = TfidfVectorizer(stop_words='english')

# fit bag of words model on our corpus
vectors = vectorizer.fit_transform(X)
y.shape,  vectors.shape

((1813,), (2930, 1582))

In [112]:
model = MultinomialNB()
model.fit(X, y) 


MultinomialNB()

In [102]:
lr_pipeline.fit(X, y)


Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('pca', PCA(n_components=0.99, svd_solver='full')),
                ('gaussiannb', GaussianNB())])

In [103]:
lr_pipeline.score(X,y)

0.5377826806398235

In [113]:
new_lyrics = ["way down we go"]
# we have to apply the same steps

# I skipped over the preprocessing 

# used only transform 

new_vectors = vectorizer.transform(new_lyrics)


In [114]:
new_vectors.todense()


matrix([[0., 0., 0., ..., 0., 0., 0.]])

In [115]:

model.predict(new_vectors)


/home/kith/.local/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MultinomialNB was fitted with feature names
  warnings.warn(


ValueError: X has 1582 features, but MultinomialNB is expecting 2930 features as input.

In [109]:

model.predict_proba(new_vectors)

ValueError: X has 1582 features, but MultinomialNB is expecting 2930 features as input.